In [2]:
import pandas as pd
file_path = 'Final_dataset.xlsx'
data = pd.read_excel(file_path)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
data

,Unnamed: 0,Team,Match,Match Date,Total Distance Covered,Distance Covered Speed 0-0.2 m/s,Distance Covered Speed 0.21-2 m/s,Distance Covered Speed 2.01-4 m/s,Distance Covered Speed 4.01-5.5 m/s,Distance Covered Speed 5.51-7 m/s,...,Cumulative Wins,Cumulative Losses,Cumulative Ties,High Speed Runs Attacking,High Speed Runs Defensive,Sprints Attacking,Sprints Defensive,Home/Away,Goal Difference,Result
0,0,Rizespor,Rizespor 1:2 Fenerbahce,11.09.20,111784.00,279.00,42168.00,43537.00,16792.00,7481.00,...,0,1,0,265.00,218.00,31.00,57.00,Home,-1.00,Lose
1,1,Rizespor,Kasimpasa 2:0 Rizespor,20.09.20,114153.00,276.00,39961.00,44998.00,19583.00,7853.00,...,0,2,0,237.00,261.00,57.00,39.00,Away,2.00,Win
2,2,Rizespor,Rizespor 1:1 Alanyaspor,27.09.20,109381.00,355.00,42605.00,41928.00,16425.00,6991.00,...,0,2,1,287.00,233.00,39.00,24.00,Home,0.00,Tie
3,3,Rizespor,Rizespor 1:0 Kayserispor,01.11.20,110355.00,232.00,43845.00,44061.00,15360.00,5918.00,...,1,2,1,252.00,182.00,25.00,37.00,Home,1.00,Win
4,4,Rizespor,Hatayspor 2:2 Rizespor,22.11.20,114306.00,297.00,41501.00,44260.00,17930.00,8198.00,...,1,2,2,320.00,211.00,58.00,52.00,Away,0.00,Tie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,709,Konyaspor,Konyaspor 5:1 Fatih Karagumruk,02.05.21,108085.00,332.00,41587.00,39416.00,17449.00,7855.00,...,11,14,10,330.00,197.00,18.00,42.00,Home,4.00,Win
710,710,Konyaspor,Goztepe 0:1 Konyaspor,08.05.21,109400.00,320.00,38555.00,42085.00,18700.00,8391.00,...,12,14,10,356.00,222.00,26.00,47.00,Away,-1.00,Lose
711,711,Konyaspor,Antalyaspor 0:0 Konyaspor,15.05.21,109703.00,297.00,40928.00,43076.00,16551.00,7605.00,...,12,14,11,295.00,233.00,29.00,31.00,Away,0.00,Tie
712,712,Konyaspor,On average,27.09.20 - 15.05.21,112188.89,328.51,41632.73,42781.86,18370.19,7726.19,...,0,0,0,134.58,95.99,20.21,17.44,Away,NaN,Tie


In [3]:
data.columns.tolist()

['Unnamed: 0',
 'Team',
 'Match',
 'Match Date',
 'Total Distance Covered',
 'Distance Covered Speed 0-0.2 m/s',
 'Distance Covered Speed 0.21-2 m/s',
 'Distance Covered Speed 2.01-4 m/s',
 'Distance Covered Speed 4.01-5.5 m/s',
 'Distance Covered Speed 5.51-7 m/s',
 'Distance Covered Speed >7 m/s',
 'No. of Accelerations 5.5-7 m/s²',
 'No. of Accelerations >7 m/s²',
 'Maximal Speed',
 'Home Team',
 'Home Score',
 'Away Team',
 'Away Score',
 'Cumulative Points',
 'Cumulative Goal Difference',
 'Cumulative Goals Scored',
 'Cumulative Goals Conceded',
 'Cumulative Wins',
 'Cumulative Losses',
 'Cumulative Ties',
 'High Speed Runs Attacking',
 'High Speed Runs Defensive',
 'Sprints Attacking',
 'Sprints Defensive',
 'Home/Away',
 'Goal Difference',
 'Result']